In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model

In [3]:
test_data = pd.read_csv('test_data.csv')

In [4]:
test_data

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,4,2022-06-20 10:24:38,7,191,26000000.0,11.4,NaN,209531,1978.0,0.0,...,EARNEDINCOME,20100301.0,정규직,기타가족소유,16000000.0,생활비,0.0,NaN,5.0,67000000.0
1,4,2022-06-20 10:24:38,7,220,10000000.0,11.4,NaN,209531,1978.0,0.0,...,EARNEDINCOME,20100301.0,정규직,기타가족소유,16000000.0,생활비,0.0,NaN,5.0,67000000.0
2,8,2022-06-02 13:10:29,30,85,13000000.0,15.9,NaN,675741,1983.0,1.0,...,PRIVATEBUSINESS,201906.0,기타,전월세,5000000.0,사업자금,0.0,NaN,5.0,50000000.0
3,8,2022-06-02 13:10:30,55,159,5000000.0,19.9,NaN,675741,1983.0,1.0,...,PRIVATEBUSINESS,201906.0,기타,전월세,5000000.0,사업자금,0.0,NaN,5.0,50000000.0
4,8,2022-06-02 13:10:29,35,29,5000000.0,14.5,NaN,675741,1983.0,1.0,...,PRIVATEBUSINESS,201906.0,기타,전월세,5000000.0,사업자금,0.0,NaN,5.0,50000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255249,2167778,2022-06-21 22:27:44,42,258,3000000.0,14.4,NaN,340207,1989.0,1.0,...,EARNEDINCOME2,20210901.0,계약직,자가,10000000.0,생활비,0.0,NaN,8.0,328000000.0
3255250,2167791,2022-06-24 12:12:42,35,29,5000000.0,12.8,NaN,485732,1971.0,1.0,...,PRIVATEBUSINESS,20210216.0,기타,기타가족소유,20000000.0,사업자금,0.0,NaN,4.0,37000000.0
3255251,2167822,2022-06-29 14:08:35,10,149,10000000.0,15.6,NaN,105164,1996.0,1.0,...,EARNEDINCOME,20220609.0,기타,기타가족소유,7000000.0,생활비,0.0,0.0,2.0,6000000.0
3255252,2167822,2022-06-29 14:08:35,10,65,11000000.0,15.6,NaN,105164,1996.0,1.0,...,EARNEDINCOME,20220609.0,기타,기타가족소유,7000000.0,생활비,0.0,0.0,2.0,6000000.0


In [5]:
test_data.dtypes

application_id                           int64
loanapply_insert_time                   object
bank_id                                  int64
product_id                               int64
loan_limit                             float64
loan_rate                              float64
is_applied                             float64
user_id                                  int64
birth_year                             float64
gender                                 float64
insert_time                             object
credit_score                           float64
yearly_income                          float64
income_type                             object
company_enter_month                    float64
employment_type                         object
houseown_type                           object
desired_amount                         float64
purpose                                 object
personal_rehabilitation_yn             float64
personal_rehabilitation_complete_yn    float64
existing_loan

In [6]:
test_data.isna().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                  60
loan_rate                                   60
is_applied                             3255254
user_id                                      0
birth_year                               36453
gender                                   36453
insert_time                                  0
credit_score                            265312
yearly_income                                6
income_type                                  0
company_enter_month                      96494
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn               15461
personal_rehabilitation_complete_yn    2560090
existing_loan

In [7]:
test_data['purpose'].replace({'생활비':'LIVING'},inplace = True)
test_data['purpose'].replace({'대환대출':'SWITCHLOAN'},inplace = True)
test_data['purpose'].replace({'사업자금':'BUSINESS'},inplace = True)
test_data['purpose'].replace({'기타':'ETC'},inplace = True)
test_data['purpose'].replace({'전월세보증금':'HOUSEDEPOSIT'},inplace = True)
test_data['purpose'].replace({'주택구입':'BUYHOUSE'},inplace = True)
test_data['purpose'].replace({'투자':'INVEST'},inplace = True)
test_data['purpose'].replace({'자동차구입':'BUYCAR'},inplace = True)
test_data['loan_limit'].fillna(test_data['loan_limit'].mean(),inplace=True)
test_data['loan_rate'].fillna(test_data['loan_rate'].mean(),inplace=True)
test_data['yearly_income'].fillna(test_data['yearly_income'].mean(),inplace=True)
test_data

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,4,2022-06-20 10:24:38,7,191,26000000.0,11.4,NaN,209531,1978.0,0.0,...,EARNEDINCOME,20100301.0,정규직,기타가족소유,16000000.0,LIVING,0.0,NaN,5.0,67000000.0
1,4,2022-06-20 10:24:38,7,220,10000000.0,11.4,NaN,209531,1978.0,0.0,...,EARNEDINCOME,20100301.0,정규직,기타가족소유,16000000.0,LIVING,0.0,NaN,5.0,67000000.0
2,8,2022-06-02 13:10:29,30,85,13000000.0,15.9,NaN,675741,1983.0,1.0,...,PRIVATEBUSINESS,201906.0,기타,전월세,5000000.0,BUSINESS,0.0,NaN,5.0,50000000.0
3,8,2022-06-02 13:10:30,55,159,5000000.0,19.9,NaN,675741,1983.0,1.0,...,PRIVATEBUSINESS,201906.0,기타,전월세,5000000.0,BUSINESS,0.0,NaN,5.0,50000000.0
4,8,2022-06-02 13:10:29,35,29,5000000.0,14.5,NaN,675741,1983.0,1.0,...,PRIVATEBUSINESS,201906.0,기타,전월세,5000000.0,BUSINESS,0.0,NaN,5.0,50000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255249,2167778,2022-06-21 22:27:44,42,258,3000000.0,14.4,NaN,340207,1989.0,1.0,...,EARNEDINCOME2,20210901.0,계약직,자가,10000000.0,LIVING,0.0,NaN,8.0,328000000.0
3255250,2167791,2022-06-24 12:12:42,35,29,5000000.0,12.8,NaN,485732,1971.0,1.0,...,PRIVATEBUSINESS,20210216.0,기타,기타가족소유,20000000.0,BUSINESS,0.0,NaN,4.0,37000000.0
3255251,2167822,2022-06-29 14:08:35,10,149,10000000.0,15.6,NaN,105164,1996.0,1.0,...,EARNEDINCOME,20220609.0,기타,기타가족소유,7000000.0,LIVING,0.0,0.0,2.0,6000000.0
3255252,2167822,2022-06-29 14:08:35,10,65,11000000.0,15.6,NaN,105164,1996.0,1.0,...,EARNEDINCOME,20220609.0,기타,기타가족소유,7000000.0,LIVING,0.0,0.0,2.0,6000000.0


In [8]:
test_data.isna().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255254
user_id                                      0
birth_year                               36453
gender                                   36453
insert_time                                  0
credit_score                            265312
yearly_income                                0
income_type                                  0
company_enter_month                      96494
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn               15461
personal_rehabilitation_complete_yn    2560090
existing_loan

In [9]:
test_data['birth_year'].fillna(test_data['birth_year'].mean(),inplace=True)
test_data['gender'].fillna(0 ,inplace=True)

In [10]:
test_data.isna().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255254
user_id                                      0
birth_year                                   0
gender                                       0
insert_time                                  0
credit_score                            265312
yearly_income                                0
income_type                                  0
company_enter_month                      96494
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn               15461
personal_rehabilitation_complete_yn    2560090
existing_loan

In [11]:
df = test_data.copy()

In [12]:
df['company_enter_month'] = pd.to_datetime(df['company_enter_month'], format= '%Y-%m-%d %H:%M:%S')
df['company_enter_month_year'] = df['company_enter_month'].dt.year
df['company_enter_month_month'] = df['company_enter_month'].dt.month
df.drop(columns=['company_enter_month'], inplace=True)

In [14]:
df.isna().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255254
user_id                                      0
birth_year                                   0
gender                                       0
insert_time                                  0
credit_score                            265312
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn               15461
personal_rehabilitation_complete_yn    2560090
existing_loan_cnt                       541657
existing_loan

In [15]:
df['loanapply_insert_time'] = pd.to_datetime(df['loanapply_insert_time'], format= '%Y-%m-%d %H:%M:%S')
df['insert_time'] = pd.to_datetime(df['insert_time'], format= '%Y-%m-%d %H:%M:%S')

df['loanapply_insert_year'] = df['loanapply_insert_time'].dt.year
df['loanapply_insert_month'] = df['loanapply_insert_time'].dt.month
df['loanapply_insert_day'] = df['loanapply_insert_time'].dt.day
df['loanapply_insert_hour'] = df['loanapply_insert_time'].dt.hour
df['loanapply_insert_minute'] = df['loanapply_insert_time'].dt.minute

df['insert_time_year'] = df['insert_time'].dt.year
df['insert_time_month'] = df['insert_time'].dt.month
df['insert_time_day'] = df['insert_time'].dt.day
df['insert_time_hour'] = df['insert_time'].dt.hour
df['insert_time_minute'] = df['insert_time'].dt.minute

df.drop(columns=['loanapply_insert_time'], inplace=True)
df.drop(columns=['insert_time'], inplace=True)
df

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,credit_score,...,loanapply_insert_year,loanapply_insert_month,loanapply_insert_day,loanapply_insert_hour,loanapply_insert_minute,insert_time_year,insert_time_month,insert_time_day,insert_time_hour,insert_time_minute
0,4,7,191,26000000.0,11.4,NaN,209531,1978.0,0.0,720.0,...,2022,6,20,10,24,2022,6,20,10,24
1,4,7,220,10000000.0,11.4,NaN,209531,1978.0,0.0,720.0,...,2022,6,20,10,24,2022,6,20,10,24
2,8,30,85,13000000.0,15.9,NaN,675741,1983.0,1.0,590.0,...,2022,6,2,13,10,2022,6,2,13,10
3,8,55,159,5000000.0,19.9,NaN,675741,1983.0,1.0,590.0,...,2022,6,2,13,10,2022,6,2,13,10
4,8,35,29,5000000.0,14.5,NaN,675741,1983.0,1.0,590.0,...,2022,6,2,13,10,2022,6,2,13,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255249,2167778,42,258,3000000.0,14.4,NaN,340207,1989.0,1.0,700.0,...,2022,6,21,22,27,2022,6,21,22,27
3255250,2167791,35,29,5000000.0,12.8,NaN,485732,1971.0,1.0,680.0,...,2022,6,24,12,12,2022,6,24,12,12
3255251,2167822,10,149,10000000.0,15.6,NaN,105164,1996.0,1.0,570.0,...,2022,6,29,14,8,2022,6,29,14,8
3255252,2167822,10,65,11000000.0,15.6,NaN,105164,1996.0,1.0,570.0,...,2022,6,29,14,8,2022,6,29,14,8


In [16]:
df.isna().sum()

application_id                               0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255254
user_id                                      0
birth_year                                   0
gender                                       0
credit_score                            265312
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn               15461
personal_rehabilitation_complete_yn    2560090
existing_loan_cnt                       541657
existing_loan_amt                       845679
company_enter_month_year                 96494
company_enter

In [17]:
df.dtypes

application_id                           int64
bank_id                                  int64
product_id                               int64
loan_limit                             float64
loan_rate                              float64
is_applied                             float64
user_id                                  int64
birth_year                             float64
gender                                 float64
credit_score                           float64
yearly_income                          float64
income_type                             object
employment_type                         object
houseown_type                           object
desired_amount                         float64
purpose                                 object
personal_rehabilitation_yn             float64
personal_rehabilitation_complete_yn    float64
existing_loan_cnt                      float64
existing_loan_amt                      float64
company_enter_month_year               float64
company_enter

In [22]:
col_list = list(df.columns)
col_list
obj_list = [c for c in col_list if str(df[c].dtype) == 'object']
obj_list

['income_type', 'employment_type', 'houseown_type', 'purpose']

In [23]:
for obj in obj_list:
  print(df[obj].value_counts())

EARNEDINCOME       2750328
PRACTITIONER        128579
PRIVATEBUSINESS     121961
OTHERINCOME          96792
EARNEDINCOME2        92748
FREELANCER           64846
Name: income_type, dtype: int64
정규직    2665807
계약직     287697
기타      268184
일용직      33566
Name: employment_type, dtype: int64
전월세       1429221
자가         940401
기타가족소유     885095
배우자           537
Name: houseown_type, dtype: int64
LIVING          1670881
SWITCHLOAN      1006932
HOUSEDEPOSIT     142623
BUYHOUSE         136586
BUSINESS         133268
ETC               96175
INVEST            65977
BUYCAR             2812
Name: purpose, dtype: int64


In [24]:
ohe = OneHotEncoder(sparse=False)
for c in obj_list:
  converted = df[c].values.reshape(-1, 1)
  ohe.fit(converted)
  transformed_feature = ohe.transform(converted)

  encoded_cols = [f'{c}_{i}' for i in range(0, transformed_feature.shape[1])]
  encoded_df = pd.DataFrame(ohe.transform(df[[c]]), columns=encoded_cols)
  encoded_df = encoded_df.astype(int)

  df.drop(c, axis=1, inplace=True)
  df = pd.concat([df, encoded_df], axis=1)

  col_list.remove(c)
  col_list += encoded_cols

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [25]:
df.isna().sum()

application_id                               0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255254
user_id                                      0
birth_year                                   0
gender                                       0
credit_score                            265312
yearly_income                                0
desired_amount                               0
personal_rehabilitation_yn               15461
personal_rehabilitation_complete_yn    2560090
existing_loan_cnt                       541657
existing_loan_amt                       845679
company_enter_month_year                 96494
company_enter_month_month                96494
loanapply_insert_year                        0
loanapply_insert_month                       0
loanapply_insert_day                         0
loanapply_ins

In [26]:
df

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,credit_score,...,houseown_type_2,houseown_type_3,purpose_0,purpose_1,purpose_2,purpose_3,purpose_4,purpose_5,purpose_6,purpose_7
0,4,7,191,26000000.0,11.4,NaN,209531,1978.0,0.0,720.0,...,0,0,0,0,0,0,0,0,1,0
1,4,7,220,10000000.0,11.4,NaN,209531,1978.0,0.0,720.0,...,0,0,0,0,0,0,0,0,1,0
2,8,30,85,13000000.0,15.9,NaN,675741,1983.0,1.0,590.0,...,0,1,1,0,0,0,0,0,0,0
3,8,55,159,5000000.0,19.9,NaN,675741,1983.0,1.0,590.0,...,0,1,1,0,0,0,0,0,0,0
4,8,35,29,5000000.0,14.5,NaN,675741,1983.0,1.0,590.0,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255249,2167778,42,258,3000000.0,14.4,NaN,340207,1989.0,1.0,700.0,...,1,0,0,0,0,0,0,0,1,0
3255250,2167791,35,29,5000000.0,12.8,NaN,485732,1971.0,1.0,680.0,...,0,0,1,0,0,0,0,0,0,0
3255251,2167822,10,149,10000000.0,15.6,NaN,105164,1996.0,1.0,570.0,...,0,0,0,0,0,0,0,0,1,0
3255252,2167822,10,65,11000000.0,15.6,NaN,105164,1996.0,1.0,570.0,...,0,0,0,0,0,0,0,0,1,0


In [27]:
df['personal_rehabilitation_yn'].fillna(2,inplace=True)
df.loc[df['personal_rehabilitation_yn']==0,'personal_rehabilitation_complete_yn']=2
df.loc[df['personal_rehabilitation_yn']==2,'personal_rehabilitation_complete_yn']=3

df.loc[df['personal_rehabilitation_yn']==0,'personal_rehabilitation_yn']= '0'
df.loc[df['personal_rehabilitation_yn']==1,'personal_rehabilitation_yn']= '1'
df.loc[df['personal_rehabilitation_yn']==2,'personal_rehabilitation_yn']= '2'
df.loc[df['personal_rehabilitation_complete_yn']==0,'personal_rehabilitation_complete_yn']= '0'
df.loc[df['personal_rehabilitation_complete_yn']==1,'personal_rehabilitation_complete_yn']= '1'
df.loc[df['personal_rehabilitation_complete_yn']==2,'personal_rehabilitation_complete_yn']= '2'
df.loc[df['personal_rehabilitation_complete_yn']==3,'personal_rehabilitation_complete_yn']= '3'

In [30]:
col_list = list(df.columns)
col_list
obj_list = [c for c in col_list if str(df[c].dtype) == 'object']
obj_list

['personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn']

In [31]:
for obj in obj_list:
  print(df[obj].value_counts())

0    3221636
1      18157
2      15461
Name: personal_rehabilitation_yn, dtype: int64
2    3221636
0      15624
3      15461
1       2533
Name: personal_rehabilitation_complete_yn, dtype: int64


In [32]:
ohe = OneHotEncoder(sparse=False)
for c in obj_list:
  converted = df[c].values.reshape(-1, 1)
  ohe.fit(converted)
  transformed_feature = ohe.transform(converted)

  encoded_cols = [f'{c}_{i}' for i in range(0, transformed_feature.shape[1])]
  encoded_df = pd.DataFrame(ohe.transform(df[[c]]), columns=encoded_cols)
  encoded_df = encoded_df.astype(int)

  df.drop(c, axis=1, inplace=True)
  df = pd.concat([df, encoded_df], axis=1)

  col_list.remove(c)
  col_list += encoded_cols

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [33]:
df

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,credit_score,...,purpose_5,purpose_6,purpose_7,personal_rehabilitation_yn_0,personal_rehabilitation_yn_1,personal_rehabilitation_yn_2,personal_rehabilitation_complete_yn_0,personal_rehabilitation_complete_yn_1,personal_rehabilitation_complete_yn_2,personal_rehabilitation_complete_yn_3
0,4,7,191,26000000.0,11.4,NaN,209531,1978.0,0.0,720.0,...,0,1,0,1,0,0,0,0,1,0
1,4,7,220,10000000.0,11.4,NaN,209531,1978.0,0.0,720.0,...,0,1,0,1,0,0,0,0,1,0
2,8,30,85,13000000.0,15.9,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
3,8,55,159,5000000.0,19.9,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
4,8,35,29,5000000.0,14.5,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255249,2167778,42,258,3000000.0,14.4,NaN,340207,1989.0,1.0,700.0,...,0,1,0,1,0,0,0,0,1,0
3255250,2167791,35,29,5000000.0,12.8,NaN,485732,1971.0,1.0,680.0,...,0,0,0,1,0,0,0,0,1,0
3255251,2167822,10,149,10000000.0,15.6,NaN,105164,1996.0,1.0,570.0,...,0,1,0,1,0,0,0,0,1,0
3255252,2167822,10,65,11000000.0,15.6,NaN,105164,1996.0,1.0,570.0,...,0,1,0,1,0,0,0,0,1,0


In [34]:
df.isna().sum()

application_id                                 0
bank_id                                        0
product_id                                     0
loan_limit                                     0
loan_rate                                      0
is_applied                               3255254
user_id                                        0
birth_year                                     0
gender                                         0
credit_score                              265312
yearly_income                                  0
desired_amount                                 0
existing_loan_cnt                         541657
existing_loan_amt                         845679
company_enter_month_year                   96494
company_enter_month_month                  96494
loanapply_insert_year                          0
loanapply_insert_month                         0
loanapply_insert_day                           0
loanapply_insert_hour                          0
loanapply_insert_min

In [75]:
def change_dtype(df):
    for c in df.columns:
        if 'int64' == str(df[c].dtype):
            df[c] = df[c].astype(np.int32)
        elif 'float64' == str(df[c].dtype):
            df[c] = df[c].astype(np.float32)

In [36]:
df.dtypes

application_id                             int64
bank_id                                    int64
product_id                                 int64
loan_limit                               float64
loan_rate                                float64
is_applied                               float64
user_id                                    int64
birth_year                               float64
gender                                   float64
credit_score                             float64
yearly_income                            float64
desired_amount                           float64
existing_loan_cnt                        float64
existing_loan_amt                        float64
company_enter_month_year                 float64
company_enter_month_month                float64
loanapply_insert_year                      int64
loanapply_insert_month                     int64
loanapply_insert_day                       int64
loanapply_insert_hour                      int64
loanapply_insert_min

In [76]:
change_dtype(df)
df.dtypes

application_id                             int32
bank_id                                    int32
product_id                                 int32
loan_limit                               float32
loan_rate                                float32
is_applied                               float32
user_id                                    int32
birth_year                               float32
gender                                   float32
credit_score                             float32
yearly_income                            float32
desired_amount                           float32
existing_loan_cnt                        float32
existing_loan_amt                        float32
company_enter_month_year                 float32
company_enter_month_month                float32
loanapply_insert_year                      int32
loanapply_insert_month                     int32
loanapply_insert_day                       int32
loanapply_insert_hour                      int32
loanapply_insert_min

In [38]:
df.to_csv('test_data.csv')

# -------------------------------------------------------------------------

In [2]:
df = pd.read_csv('test_data.csv')

In [5]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
df

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,credit_score,...,purpose_5,purpose_6,purpose_7,personal_rehabilitation_yn_0,personal_rehabilitation_yn_1,personal_rehabilitation_yn_2,personal_rehabilitation_complete_yn_0,personal_rehabilitation_complete_yn_1,personal_rehabilitation_complete_yn_2,personal_rehabilitation_complete_yn_3
0,4,7,191,26000000.0,11.4,NaN,209531,1978.0,0.0,720.0,...,0,1,0,1,0,0,0,0,1,0
1,4,7,220,10000000.0,11.4,NaN,209531,1978.0,0.0,720.0,...,0,1,0,1,0,0,0,0,1,0
2,8,30,85,13000000.0,15.9,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
3,8,55,159,5000000.0,19.9,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
4,8,35,29,5000000.0,14.5,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255249,2167778,42,258,3000000.0,14.4,NaN,340207,1989.0,1.0,700.0,...,0,1,0,1,0,0,0,0,1,0
3255250,2167791,35,29,5000000.0,12.8,NaN,485732,1971.0,1.0,680.0,...,0,0,0,1,0,0,0,0,1,0
3255251,2167822,10,149,10000000.0,15.6,NaN,105164,1996.0,1.0,570.0,...,0,1,0,1,0,0,0,0,1,0
3255252,2167822,10,65,11000000.0,15.6,NaN,105164,1996.0,1.0,570.0,...,0,1,0,1,0,0,0,0,1,0


In [7]:
df.isna().sum()

application_id                                 0
bank_id                                        0
product_id                                     0
loan_limit                                     0
loan_rate                                      0
is_applied                               3255254
user_id                                        0
birth_year                                     0
gender                                         0
credit_score                              265312
yearly_income                                  0
desired_amount                                 0
existing_loan_cnt                         541657
existing_loan_amt                         845679
company_enter_month_year                   96494
company_enter_month_month                  96494
loanapply_insert_year                          0
loanapply_insert_month                         0
loanapply_insert_day                           0
loanapply_insert_hour                          0
loanapply_insert_min

In [77]:
df2 = df.copy()
df2['existing_loan_cnt'].fillna(df2['existing_loan_cnt'].mean(),inplace=True)
df2['existing_loan_amt'].fillna(df2['existing_loan_amt'].mean(),inplace=True)
df2['company_enter_month_year'].fillna(df2['company_enter_month_year'].mean(),inplace=True)
df2['company_enter_month_month'].fillna(df2['company_enter_month_month'].mean(),inplace=True)
pd.isnull(df2).sum()

application_id                                 0
bank_id                                        0
product_id                                     0
loan_limit                                     0
loan_rate                                      0
is_applied                               3255254
user_id                                        0
birth_year                                     0
gender                                         0
credit_score                              265312
yearly_income                                  0
desired_amount                                 0
existing_loan_cnt                              0
existing_loan_amt                              0
company_enter_month_year                       0
company_enter_month_month                      0
loanapply_insert_year                          0
loanapply_insert_month                         0
loanapply_insert_day                           0
loanapply_insert_hour                          0
loanapply_insert_min

In [87]:
df2

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,credit_score,...,purpose_5,purpose_6,purpose_7,personal_rehabilitation_yn_0,personal_rehabilitation_yn_1,personal_rehabilitation_yn_2,personal_rehabilitation_complete_yn_0,personal_rehabilitation_complete_yn_1,personal_rehabilitation_complete_yn_2,personal_rehabilitation_complete_yn_3
0,4,7,191,26000000.0,11.400000,NaN,209531,1978.0,0.0,720.0,...,0,1,0,1,0,0,0,0,1,0
1,4,7,220,10000000.0,11.400000,NaN,209531,1978.0,0.0,720.0,...,0,1,0,1,0,0,0,0,1,0
2,8,30,85,13000000.0,15.900000,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
3,8,55,159,5000000.0,19.900000,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
4,8,35,29,5000000.0,14.500000,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255249,2167778,42,258,3000000.0,14.400000,NaN,340207,1989.0,1.0,700.0,...,0,1,0,1,0,0,0,0,1,0
3255250,2167791,35,29,5000000.0,12.800000,NaN,485732,1971.0,1.0,680.0,...,0,0,0,1,0,0,0,0,1,0
3255251,2167822,10,149,10000000.0,15.600000,NaN,105164,1996.0,1.0,570.0,...,0,1,0,1,0,0,0,0,1,0
3255252,2167822,10,65,11000000.0,15.600000,NaN,105164,1996.0,1.0,570.0,...,0,1,0,1,0,0,0,0,1,0


In [89]:
lin_reg = linear_model.LinearRegression()
X = df.dropna(axis=0)[['bank_id', 'product_id', 'loan_limit', 'loan_rate', 'birth_year', 'gender', 'insert_time_year',
                       'insert_time_month', 'insert_time_day', 'loanapply_insert_hour', 'loanapply_insert_minute', 
                      'yearly_income', 'company_enter_month_year', 'company_enter_month_month', 'desired_amount', 
                      'existing_loan_cnt', 'existing_loan_amt', 'income_type_0', 'income_type_1', 'income_type_2'
                       , 'income_type_3', 'income_type_4', 'income_type_5', 'employment_type_0', 'employment_type_1'
                       , 'employment_type_2', 'employment_type_3', 'houseown_type_0', 'houseown_type_1', 'houseown_type_2'
                       , 'houseown_type_3', 'purpose_0', 'purpose_1', 'purpose_2', 'purpose_3', 'purpose_4', 'purpose_5'
                       , 'purpose_6', 'purpose_7', 'insert_time_year', 'insert_time_month', 'insert_time_day', 'insert_time_hour'
                       , 'insert_time_minute', 'personal_rehabilitation_yn_0', 'personal_rehabilitation_yn_1'
                       , 'personal_rehabilitation_yn_2', 'personal_rehabilitation_complete_yn_0'
                       , 'personal_rehabilitation_complete_yn_1', 'personal_rehabilitation_complete_yn_2'
                       , 'personal_rehabilitation_complete_yn_3']]
y = df.dropna(axis=0)['credit_score']

In [90]:
X.shape

(0, 51)

In [84]:
y.shape

(3255254,)

In [85]:
lin_reg_model = lin_reg.fit(X, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
y_pred = lin_reg_model.predict(df2.loc[:, ['bank_id', 'product_id', 'loan_limit', 'loan_rate', 'birth_year', 'gender', 'insert_time_year',
                       'insert_time_month', 'insert_time_day', 'loanapply_insert_hour', 'loanapply_insert_minute', 
                      'yearly_income', 'company_enter_month_year', 'company_enter_month_month', 'desired_amount', 
                      'existing_loan_cnt', 'existing_loan_amt', 'income_type_0', 'income_type_1', 'income_type_2'
                       , 'income_type_3', 'income_type_4', 'income_type_5', 'employment_type_0', 'employment_type_1'
                       , 'employment_type_2', 'employment_type_3', 'houseown_type_0', 'houseown_type_1', 'houseown_type_2'
                       , 'houseown_type_3', 'purpose_0', 'purpose_1', 'purpose_2', 'purpose_3', 'purpose_4', 'purpose_5'
                       , 'purpose_6', 'purpose_7', 'insert_time_year', 'insert_time_month', 'insert_time_day', 'insert_time_hour'
                       , 'insert_time_minute', 'personal_rehabilitation_yn_0', 'personal_rehabilitation_yn_1'
                       , 'personal_rehabilitation_yn_2', 'personal_rehabilitation_complete_yn_0'
                       , 'personal_rehabilitation_complete_yn_1', 'personal_rehabilitation_complete_yn_2'
                       , 'personal_rehabilitation_complete_yn_3']]).round(1)

y_pred